In [ ]:
import numpy as np

# Constants
sigma = 0.001
beta = 100
lamda = 0.6
alpha = 0.1
v0 = 0.01
vg = 0
psi = np.inf
D = 0.00001

t0 = 0.0
tf = 5000
steps = 100000

In [ ]:
def mu_x(vec,t): 
    x,z,phi = vec
    dxdt    =       alpha*np.exp(z)*np.cos(x-t)       + v0*np.sin(phi)  + sigma*(beta+z)
    return dxdt

def mu_z(vec,t):
    x,z,phi = vec
    dzdt    =       alpha*np.exp(z)*np.sin(x-t)       + v0*np.cos(phi)  - vg
    return dzdt

def mu_phi(vec,t):
    x,z,phi = vec
    dphidt  = lamda*alpha*np.exp(z)*np.cos(x-t+2*phi) - 1/(2*psi)*np.sin(phi)    + sigma/2*(1+lamda*np.cos(2*phi))
    return dphidt

def d_mu_x(vec,t):
    x,z,phi = vec
    d2xdtdx = -alpha*np.exp(z)*np.sin(x-t)
    return d2xdtdx

def d_mu_z(vec,t): 
    x,z,phi = vec
    d2zdtdz = alpha*np.exp(z)*np.sin(x-t)
    return d2zdtdz

def d_mu_phi(vec,t): 
    x,z,phi = vec
    d2phidtdphi = -lamda*alpha*np.exp(z)*np.sin(x-t+2*phi)*2 + 1/(2*psi)*np.cos(phi)    - sigma*lamda*np.sin(2*phi)
    return d2phidtdphi

def fokker_planck(vec,t):
    x,z,phi = vec

    d2x_dtdx     = d_mu_x(vec)
    d2z_dtdz     = d_mu_z(vec)
    d2phi_dtdphi = d_mu_phi(vec)

    dxdt         = mu_x(vec)
    dzdt         = mu_z(vec)
    dphidt       = mu_phi(vec)

    f       = 0
    dx      = None
    dz      = None
    dphi    = None
    dt      = None

    df_dx           = f(x+dx,z,phi,t)   - f(x-dx,z,phi,t)   / (2*dx)
    df_dz           = f(x,z+dz,phi,t)   - f(x,z-dz,phi,t)   / (2*dz)
    df_dphi         = f(x,z,phi+dphi,t) - f(x,z,phi-dphi,t) / (2*dphi)

    d2f_dphidphi    = f(x,z,phi+dphi,t) - 2*f(x,z,phi,t) + f(x,z,phi-dphi,t) / (dphi**2)

    rhs = - (dxdt*df_dx + dzdt*df_dz + dphidt*df_dphi) - (d2x_dtdx + d2z_dtdz + d2phi_dtdphi)*f + 0.5 * (var(vec,t)**2) * (d2f_dphidphi)

    lhs = f(x,z,phi,t+dt) - f(x,z,phi,t) / dt

    return 0


In [ ]:
# Explicit Scheme

def fokker_planck_step(f,vec,t, i,j,k,n):

    dx      = vec[i+1,j,k] - vec[i,j,k]
    dz      = vec[i,j+1,k] - vec[i,j,k]
    dphi    = vec[i,j,k+1] - vec[i,j,k]
    dt      = t[n+1] - t[n]

    F_x = np.zeros_like(f)
    F_z = np.zeros_like(f)
    J_phi = np.zeros_like(f)
    grad_flux = np.zeros_like(f)

    F_x[i,j,k,n] = upwind_flux_step_x(f,vec, t, i,j,k,n)
    F_z[i,j,k,n] = upwind_flux_step_z(f,vec, t, i,j,k,n)
    J_phi[i,j,k,n] = chang_cooper_flux(f, dphi, vec, t, i,j,k,n)
    grad_flux[i,j,k,n] = (F_x[i+1,j,k,n] - F_x[i,j,k,n])/dx + (F_z[i,j+1,k,n] - F_z[i,j,k,n])/dz + (J_phi[i,j,k+1,n] - J_phi[i,j,k,n])/dphi

    return grad_flux[i,j,k,n]

def fokker_planck_time_integrator(vec0, t, i,j,k):
    f = np.zeros_like(vec0)
    for n in range(len(t)-1):
        dt = t[n+1] - t[n]
        f[i,j,k,n+1] = f[i,j,k,n] + fokker_planck_step(vec,t, i,j,k,n)*dt
    return f

def upwind_flux_step_x(f, vec, t, i,j,k,n):

    u = 0.5*(mu_x(vec[i,j,k],t[n])+mu_x(vec[i+1,j,k],t[n]))
    if u >= 0:
        F = u*(f[i,j,k,n])
    else :
        F = u*(f[i+1,j,k,n])
    return F

def upwind_flux_step_z(f, F, vec, t, i,j,k,n):

    u = 0.5*(mu_z(vec[i,j,k],t[n])+mu_z(vec[i,j+1,k],t[n]))
    if u >= 0:
        F = u*(f[i,j,k,n])
    else :
        F = u*(f[i,j+1,k,n])
    return F

def chang_cooper_flux(f, dphi, vec, t, i,j,k,n):

    A = 0.5*(mu_phi(vec[i,j,k],t[n])+mu_phi(vec[i,j,k+1],t[n]))
    D = 0.5*var(vec[i,j,k],t[n])**2 + 0.5*var(vec[i,j,k+1],t[n])**2
    
    P = A/D*dphi

    def delta(P):
        if P == 0:
            return 1
        else:
            return P/(np.exp(P)-1)
    
    delta_P = delta(P)

    J = A*( (1-delta_P)*f[i,j,k+1,n] + delta_P*f[i,j,k,n] ) - D*( f[i,j,k+1,n] - f[i,j,k,n] )/dphi

    return J
